# **Import Libraries**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

# **🚗 1. Pemahaman Dataset**

## **📌 Sumber dan Deskripsi Dataset**  

Dataset **"Car Price Prediction"** digunakan untuk memprediksi harga kendaraan berdasarkan berbagai fitur dan karakteristik mobil. Dataset ini berasal dari **Kaggle**, salah satu platform utama untuk dataset dan kompetisi machine learning. 

📥 **Link Dataset**: [Car Price Prediction - Kaggle](https://www.kaggle.com/datasets/deepcontractor/car-price-prediction-challenge)  

### **🔍 Gambaran Umum Dataset:**  
- 📊 **Jumlah Baris**: 19.237 entri  
- 📈 **Jumlah Kolom**: 18 variabel  
- 🎯 **Target**: **"Price"** (harga mobil)  
- 📅 **Rentang Tahun Produksi**: 1939 - 2020  
- 💰 **Rentang Harga**: 1 - 26.307.500 (kemungkinan dalam USD)  
- 📉 **Rata-rata Harga**: 18.555,93  

### **💡 Mengapa dataset ini cocok untuk model regresi?**
✔️ **Variabel target numerik yang jelas** (Price)  
✔️ **Berisi fitur numerik & kategorikal**  
✔️ **Jumlah data cukup besar untuk pelatihan yang baik**  
✔️ **Mewakili berbagai segmen kendaraan dengan rentang harga luas**  

---

### **🔎 Variabel yang Digunakan** 

Dataset ini memiliki **18 kolom** yang mencakup berbagai spesifikasi kendaraan:  

| 🔢 **Variabel** | 🏷 **Tipe Data** | 📝 **Deskripsi** |
|---------------|--------------|----------------|
| 🆔 **ID** | Integer | Nomor identifikasi unik untuk setiap mobil |
| 💵 **Price** | Integer | **Target** - Harga kendaraan |
| 💸 **Levy** | String | Pajak atau biaya terkait kendaraan |
| 🏭 **Manufacturer** | String | Merek mobil (contoh: LEXUS, HONDA, TOYOTA) |
| 🚘 **Model** | String | Model spesifik kendaraan |
| 📅 **Prod. year** | Integer | Tahun produksi mobil (1939-2020) |
| 🚙 **Category** | String | Tipe kendaraan (contoh: Jeep, Sedan, Hatchback) |
| 🛋 **Leather interior** | String | Interior kulit (Yes/No) |
| ⛽ **Fuel type** | String | Jenis bahan bakar (Petrol, Diesel, Hybrid, dll) |
| 🔧 **Engine volume** | String | Volume mesin dalam liter |
| 🏁 **Mileage** | String | Jarak tempuh kendaraan (biasanya dalam km) |
| 🔥 **Cylinders** | Float | Jumlah silinder mesin |
| ⚙️ **Gear box type** | String | Jenis transmisi (Automatic, Manual, dll) |
| 🚀 **Drive wheels** | String | Konfigurasi penggerak roda (4x4, Front, Rear) |
| 🚪 **Doors** | String | Jumlah pintu kendaraan |
| 🛞 **Wheel** | String | Posisi kemudi (Left-wheel, Right-hand drive) |
| 🎨 **Color** | String | Warna kendaraan |
| 🛡 **Airbags** | Integer | Jumlah airbag dalam kendaraan |

---

Dataset ini menawarkan banyak fitur yang dapat digunakan untuk eksplorasi lebih lanjut dan membangun model prediktif yang akurat. 🚀✨

💾 Selanjutnya mari kita lakukan **loading dataset** dan kita simpan kedalam sebuah **dataframe pandas** dengan nama `df`


In [2]:
df = pd.read_csv('data/raw-data/car_price_prediction.csv')

## 📊 **Statistik Deskriptif & Visualisasi Awal** 

Sebelum membangun model prediksi, kita perlu **memahami karakteristik data** melalui analisis statistik dan visualisasi.  

💡 **Apa yang akan kita lakukan?**  
✅ Meninjau distribusi harga mobil dan fitur lainnya  
✅ Menganalisis tren berdasarkan tahun produksi, kategori, dan jenis bahan bakar  
✅ Menggunakan grafik untuk melihat pola tersembunyi dalam data  

🔍 Mari kita jelajahi data ini lebih dalam dengan statistik deskriptif dan visualisasi yang menarik! 🚀📈  

---

### **🎨 Konfigurasi Gaya Visualisasi**  

Sebelum mulai membuat grafik, kita perlu **mengatur gaya visualisasi** agar hasilnya lebih **menarik, konsisten, dan mudah dibaca**.  

In [3]:
# Mengatur style visualisasi
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

📊 Dengan konfigurasi ini, visualisasi kita akan lebih **informatif dan estetis**! 🚀🎨  

### 🔍 **Beberapa Baris Pertama Dataset**  

Sebelum masuk lebih dalam, mari kita **mengintip isi dataset** untuk memahami struktur dan jenis data yang ada. 👀  

📌 Mari kita lihat bagaimana tampilan awal dataset kita! 📊  

In [4]:
df.head()

,ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,45654403,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12
1,44731507,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000 km,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8
2,45774419,8467,-,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000 km,4.0,Variator,Front,04-May,Right-hand drive,Black,2
3,45769185,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966 km,4.0,Automatic,4x4,04-May,Left wheel,White,0
4,45809263,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4



Dataset ini berisi informasi kendaraan dengan berbagai fitur seperti **merek, model, tahun produksi, jenis bahan bakar, dan harga**. 

Beberapa kolom seperti **Levy dan Mileage** memiliki format angka bercampur string yang perlu dibersihkan.  

Selain itu, dapat kita lihat terdapat fitur kategorikal seperti **Leather Interior, Gear Box Type, dan Drive Wheels**, yang penting untuk eksplorasi lebih lanjut! 🚗📊  

---

### 📂 **Informasi Dasar Dataset**  

Sebelum melangkah lebih jauh, mari kita **mengenal struktur dataset** ini dengan melihat jumlah baris, jumlah kolom, dan tipe data yang digunakan.  

In [5]:
# Informasi Dasar Dataset
print("\nInformasi Dasar Dataset:")
print(f"Jumlah Baris: {df.shape[0]}")
print(f"Jumlah Kolom: {df.shape[1]}")
print(f"Ukuran Memory: {df.memory_usage().sum() / 1024**2:.2f} MB")
print("\nTipe Data Masing-masing Kolom:")
display(df.dtypes)


Informasi Dasar Dataset:
Jumlah Baris: 19237
Jumlah Kolom: 18
Ukuran Memory: 2.64 MB

Tipe Data Masing-masing Kolom:


ID                    int64
Price                 int64
Levy                 object
Manufacturer         object
Model                object
Prod. year            int64
Category             object
Leather interior     object
Fuel type            object
Engine volume        object
Mileage              object
Cylinders           float64
Gear box type        object
Drive wheels         object
Doors                object
Wheel                object
Color                object
Airbags               int64
dtype: object

📌 **Mengenal Struktur Data dengan Lebih Dekat!**  

Dataset ini berisi **19.237 baris dan 18 kolom**, dengan ukuran yang cukup ringan (**2.64 MB**).  
Terdapat kombinasi **data numerik** 🧮 (int64, float64) dan **data kategorikal** 🔤 (object).  

🚨 **Tapi ada yang perlu diperhatikan!**  
Beberapa kolom memiliki **tipe data yang kurang sesuai**, sehingga harus kita bersihkan sebelum dianalisis lebih lanjut:  

🔍 **Kolom yang perlu diperbaiki:**  
✅ **Levy** → Harusnya **int/float**, karena pajak dan biaya kendaraan seharusnya berbentuk angka 💰  
✅ **Cylinders** → Tidak mungkin ada **jumlah silinder desimal**, harus berupa **bilangan bulat (int)** ⚙️  
✅ **Engine Volume** → Harus **float**, karena kapasitas mesin sering dinyatakan dengan desimal 🏎️  
✅ **Mileage** → Seharusnya **int/float**, karena menunjukkan jarak tempuh dalam angka 🚗  

---

### 🛠️ **Memperbaiki Struktur Data**  

Sebelum melangkah lebih jauh, kita perlu **membersihkan dan memperbaiki** beberapa kolom yang memiliki **tipe data tidak sesuai**.  

🔎 **Apa yang akan kita lakukan?**  
- **Mengonversi kolom Levy & Mileage** menjadi **numerik** agar bisa digunakan dalam analisis 📊  
- **Memastikan Cylinders bertipe integer**, karena tidak ada mobil dengan setengah silinder! ⚙️  
- **Mengubah Engine Volume menjadi float**, karena kapasitas mesin sering menggunakan desimal 🚗  

Mari mulai dengan kolom `Levy`⚡

In [6]:
df['Levy'].unique()

array(['1399', '1018', '-', '862', '446', '891', '761', '751', '394',
       '1053', '1055', '1079', '810', '2386', '1850', '531', '586',
       '1249', '2455', '583', '1537', '1288', '915', '1750', '707',
       '1077', '1486', '1091', '650', '382', '1436', '1194', '503',
       '1017', '1104', '639', '629', '919', '781', '530', '640', '765',
       '777', '779', '934', '769', '645', '1185', '1324', '830', '1187',
       '1111', '760', '642', '1604', '1095', '966', '473', '1138', '1811',
       '988', '917', '1156', '687', '11714', '836', '1347', '2866',
       '1646', '259', '609', '697', '585', '475', '690', '308', '1823',
       '1361', '1273', '924', '584', '2078', '831', '1172', '893', '1872',
       '1885', '1266', '447', '2148', '1730', '730', '289', '502', '333',
       '1325', '247', '879', '1342', '1327', '1598', '1514', '1058',
       '738', '1935', '481', '1522', '1282', '456', '880', '900', '798',
       '1277', '442', '1051', '790', '1292', '1047', '528', '1211',
       

⚠️ **Ada Penyusup di Data Kita!** 😲  

Saat kita cek tipe data, ternyata beberapa kolom yang **seharusnya numerik** malah bertipe **object**.  
Setelah diamati lebih lanjut, penyebabnya adalah adanya nilai **"-"** yang bukan merupakan angka! ❌🔢  

🎯 **Solusi!**  
Kita akan mengganti nilai **"-"** dengan **NaN (Not a Number)**, sehingga dianggap sebagai **nilai kosong**.  
Setelah itu, kita bisa dengan mudah **mengubah tipe kolom menjadi numerik** dan siap digunakan dalam analisis! 🚀📊  

In [7]:
# 1. Perbaiki kolom Levy: ganti "-" dengan NaN
df['Levy'] = df['Levy'].replace('-', np.nan)
df['Levy'] = pd.to_numeric(df['Levy'], errors='coerce')
df['Levy'].dtypes

dtype('float64')

Oke sekarang kolom `Levy` sudah bertipe numerik ✅

Lanjut ke kolom `Mileage` 🔢

In [8]:
df['Mileage'].unique()

array(['186005 km', '192000 km', '200000 km', ..., '140607 km',
       '307325 km', '186923 km'], dtype=object)

Setelah dilakukan pengecekan, ditemukan bahwa kolom **Mileage** memiliki format yang tidak sepenuhnya numerik.  
Setiap nilai pada kolom ini memiliki akhiran **"km"**, sehingga tipe data yang seharusnya numerik terbaca sebagai **object**.  

Agar dapat digunakan dalam analisis lebih lanjut, kita perlu **menghapus akhiran "km"** dan mengonversi kolom ini menjadi **numerik**.  
Langkah ini memastikan data lebih bersih dan dapat digunakan dalam perhitungan statistik serta pemodelan. 📊✅  

In [9]:
def clean_mileage(val):
    if isinstance(val, str):
        # Ekstrak angka dari string (misal: "186005 km" -> 186005)
        match = re.search(r'(\d+)', str(val))
        if match:
            return int(match.group(1))
    return np.nan

df['Mileage'] = df['Mileage'].apply(clean_mileage)

df['Mileage'].dtypes

dtype('int64')

Oke sekarang kolom `Mileage` sudah bertipe numerik ✅

Lanjut ke kolom `Cylinders` ➰

In [10]:
df['Cylinders'].unique()

array([ 6.,  4.,  8.,  1., 12.,  3.,  2., 16.,  5.,  7.,  9., 10., 14.])

Setelah dicek, kolom **Cylinders** memiliki tipe data **float**, padahal jumlah silinder pada kendaraan harusnya berupa **bilangan bulat (int)**.  
Tidak masuk akal jika sebuah kendaraan memiliki jumlah silinder desimal, sehingga kita perlu mengonversinya ke **integer** agar lebih sesuai.  

In [11]:
df["Cylinders"] = df["Cylinders"].astype(int)

df['Cylinders'].dtypes

dtype('int64')

Oke sekarang kolom `Cylinders` sudah bertipe int ✅

Lanjut ke kolom `Engine Volume` 🚒

In [12]:
df['Engine volume'].unique()

array(['3.5', '3', '1.3', '2.5', '2', '1.8', '2.4', '4', '1.6', '3.3',
       '2.0 Turbo', '2.2 Turbo', '4.7', '1.5', '4.4', '3.0 Turbo',
       '1.4 Turbo', '3.6', '2.3', '1.5 Turbo', '1.6 Turbo', '2.2',
       '2.3 Turbo', '1.4', '5.5', '2.8 Turbo', '3.2', '3.8', '4.6', '1.2',
       '5', '1.7', '2.9', '0.5', '1.8 Turbo', '2.4 Turbo', '3.5 Turbo',
       '1.9', '2.7', '4.8', '5.3', '0.4', '2.8', '3.2 Turbo', '1.1',
       '2.1', '0.7', '5.4', '1.3 Turbo', '3.7', '1', '2.5 Turbo', '2.6',
       '1.9 Turbo', '4.4 Turbo', '4.7 Turbo', '0.8', '0.2 Turbo', '5.7',
       '4.8 Turbo', '4.6 Turbo', '6.7', '6.2', '1.2 Turbo', '3.4',
       '1.7 Turbo', '6.3 Turbo', '2.7 Turbo', '4.3', '4.2', '2.9 Turbo',
       '0', '4.0 Turbo', '20', '3.6 Turbo', '0.3', '3.7 Turbo', '5.9',
       '5.5 Turbo', '0.2', '2.1 Turbo', '5.6', '6', '0.7 Turbo',
       '0.6 Turbo', '6.8', '4.5', '0.6', '7.3', '0.1', '1.0 Turbo', '6.3',
       '4.5 Turbo', '0.8 Turbo', '4.2 Turbo', '3.1', '5.0 Turbo', '6.4',
       '3

Setelah dianalisis, kolom **Engine Volume** memiliki masalah format, di mana beberapa nilai mengandung kata **"Turbo"**, menyebabkan tipe data kolom menjadi **object** alih-alih numerik.  
Padahal, informasi "Turbo" seharusnya menjadi fitur terpisah dan tidak bercampur dengan angka.  

Untuk mengatasinya, kita akan melakukan dua langkah:  
✅ **Menghapus kata "Turbo"** sehingga kolom **Engine Volume** bisa dikonversi menjadi tipe **float**.  
✅ **Menambahkan kolom baru "IsTurbo"** yang berisi nilai **True** jika mobil memiliki turbo dan **False** jika tidak.  

Dengan langkah ini, kita tidak hanya membersihkan data, tetapi juga mempertahankan informasi penting tentang turbo dalam format yang lebih terstruktur. 🔥🔧

In [13]:
# Membuat kolom IsTurbo berdasarkan apakah terdapat kata "Turbo"
df["IsTurbo"] = df["Engine volume"].str.contains("Turbo", na=False)

# Menghapus "Turbo" dan mengonversi kolom menjadi float
df["Engine volume"] = df["Engine volume"].str.replace(" Turbo", "", regex=True).astype(float)

---

### 🎯 **Statistik Deskriptif untuk Kolom Numerik**  

Sebelum melangkah lebih jauh, mari kita lihat **statistik deskriptif** dari kolom numerik! 📊  

🔎 **Apa yang kita lakukan?**  
- Menganalisis **rata-rata, median, nilai minimum & maksimum** untuk memahami distribusi data.  
- Menghitung **range (max - min)** untuk mengetahui tingkat variasi setiap fitur.  
- Melihat **jumlah nilai unik** guna membedakan fitur **kontinu atau diskrit**.  

🚀 **Mengapa ini penting?**  
- Membantu menemukan **outlier**, misalnya harga mobil yang tidak wajar.  
- Mengidentifikasi apakah ada **skala nilai yang perlu dinormalisasi**.  
- Menentukan fitur mana yang paling berpengaruh terhadap harga mobil!  

Dengan ini, kita bisa mulai menggali **wawasan menarik dari dataset**! 🔥📉  

In [14]:
# Analisis statistik deskriptif untuk kolom numerik
print("Statistik Deskriptif untuk Kolom Numerik:")
numeric_stats = df.describe().T
numeric_stats['range'] = numeric_stats['max'] - numeric_stats['min']
numeric_stats = numeric_stats.sort_values(by='range', ascending=False)
display(numeric_stats)

# Menampilkan jumlah nilai unik untuk setiap kolom numerik
print("\nJumlah Nilai Unik pada Kolom Numerik:")
for col in df.select_dtypes(include=['int64', 'float64']).columns:
    print(f"{col}: {df[col].nunique()} nilai unik")

Statistik Deskriptif untuk Kolom Numerik:


,count,mean,std,min,25%,50%,75%,max,range
Mileage,19237.0,1.532236e+06,4.840387e+07,0.0,70139.0,126000.0,188888.0,2.147484e+09,2.147484e+09
Price,19237.0,1.855593e+04,1.905813e+05,1.0,5331.0,13172.0,22075.0,2.630750e+07,2.630750e+07
ID,19237.0,4.557654e+07,9.365914e+05,20746880.0,45698374.0,45772308.0,45802036.0,4.581665e+07,2.506977e+07
Levy,13418.0,9.068381e+02,4.618671e+02,87.0,640.0,781.0,1058.0,1.171400e+04,1.162700e+04
Prod. year,19237.0,2.010913e+03,5.668673e+00,1939.0,2009.0,2012.0,2015.0,2.020000e+03,8.100000e+01
Engine volume,19237.0,2.307990e+00,8.778045e-01,0.0,1.8,2.0,2.5,2.000000e+01,2.000000e+01
Airbags,19237.0,6.582627e+00,4.320168e+00,0.0,4.0,6.0,12.0,1.600000e+01,1.600000e+01
Cylinders,19237.0,4.582991e+00,1.199933e+00,1.0,4.0,4.0,4.0,1.600000e+01,1.500000e+01



Jumlah Nilai Unik pada Kolom Numerik:
ID: 18924 nilai unik
Price: 2315 nilai unik
Levy: 558 nilai unik
Prod. year: 54 nilai unik
Engine volume: 65 nilai unik
Mileage: 7687 nilai unik
Cylinders: 13 nilai unik
Airbags: 17 nilai unik
